In [26]:
import pandas as pd
import numpy as np

In [27]:
train_data = pd.read_csv("../../train.csv")
test_data = pd.read_csv("../../test.csv")

In [28]:
#Removing cols

#removing 'cabin' column because it has many NAN cols
columns = ['Ticket', 'Name', 'Cabin', 'Ticket', 'PassengerId']
train_data = train_data.drop(columns=columns)
test_date = test_data.drop(columns=columns)

In [29]:
train_data

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S
887,1,1,female,19.0,0,0,30.0000,S
888,0,3,female,NaN,1,2,23.4500,S
889,1,1,male,26.0,0,0,30.0000,C
